In [46]:
import pandas as pd
import datetime

In [47]:
CLUBS = ['SSG', '키움','LG','KT','KIA', '롯데', '삼성', '두산', '한화', 'NC']
HOLIDAYS = {}
HOLIDAYS['20230505'] = 2
HOLIDAYS['20230527'] = 1
HOLIDAYS['20230606'] = 1
HOLIDAYS['20230815'] = 1

print(len(CLUBS))
print(len(HOLIDAYS))

10
4


## Matches, Calendar, Initial Plan

아래와 같은 형태로 필요

```
 {
      "index": 0,
      "home": "SSG",
      "away": "두산",
      "datetime": "2021-04-02 00:00:00",
      "consecutive": 3,
      "weekend" : 0,
      "holiday" : 2,

    },
```

일단 모두 저장하기

In [48]:
total_matches = []
index = 0
for club in CLUBS:
    print(club)
    count = 0
    club_matchs = pd.read_excel('./2023_schedule.xlsx', engine='openpyxl', sheet_name=club)
    
    prev_home = ''
    prev_away = ''
    
    consecutive = 0
  
    weekend = 0
    
    for line in club_matchs.iterrows():
        
        row = line[1]
        away = row['경기'].split('vs')[0]
        home = row['경기'].split('vs')[1]
        
        home_yn = True if home == club else False
        if home_yn is False: # 홈경기만 진행
            continue
        
        index +=1
            
        row['날짜']
        month = row['날짜'][:2]
        day = row['날짜'][3:5]
        day_of_week = row['날짜'][6:7]
        hh = row['시간'].hour
        mm = row['시간'].minute
        match_date = datetime.datetime(2023, int(month), int(day), int(hh), int(mm))
        stadium = row['구장']
        
        
        if match_date.strftime('%Y%m%d') in HOLIDAYS :
            print(match_date.strftime('%Y%m%d'))
            holiday_score = HOLIDAYS[match_date.strftime('%Y%m%d')]
        elif day_of_week in ['토', '일'] :
            holiday_score = 1
        else :
            holiday_score = 0

        if prev_home == home and prev_away == away :
            consecutive += 1
        else :
            consecutive = 1
            
        prev_home = home
        prev_away = away
        prev_consecutive = consecutive
        # print(day_of_week)
        # print(holiday_score)
        match_dict = {}
        match_dict['index'] = index
        match_dict['home'] = home
        match_dict['away'] = away
        match_dict['consecutive'] = consecutive
        match_dict['match_date'] = match_date
        match_dict['holiday'] = holiday_score
        match_dict['day_of_week'] = day_of_week
        
        total_matches.append(match_dict)

SSG
키움
20230505
20230527
20230606
LG
KT
KIA
20230527
20230606
20230815
롯데
20230505
20230606
20230815
삼성
20230527
20230606
20230815
두산
20230505
20230527
20230606
20230815
한화
20230505
NC
20230505
20230527
20230815


저장 데이터 확인하기

In [49]:
import csv
myFile = open('total_matches.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
for row in total_matches:
    writer.writerow(row.values())
myFile.close()

In [51]:

# ```
#  {
#       "index": 0,
#       "home": "SSG",
#       "away": "두산",
#       "datetime": "2021-04-02 00:00:00",
#       "consecutive": 3,
#       "weekend" : 0,
#       "holiday" : 2,

#     },
# ```
tot_matches_summary = []
index = 0
for k, current_row in enumerate(total_matches):
    # print(index)
    if index >= len(total_matches):
        break
    
    if index == len(total_matches)  -1 or  total_matches[k + 1]['consecutive'] == 1:
        consecutive = current_row['consecutive']
        start_row_index = k - consecutive + 1
        start_row = total_matches[start_row_index]
        
        datetime = start_row['match_date']
        day_of_week = start_row['day_of_week']
        
        home = start_row['home']
        away = start_row['away']
        
        weekend = 0 if start_row['day_of_week'] == '화' else 1 
        holiday = 0
        for i in range(start_row_index, k + 1):
            holiday += total_matches[i]['holiday']
        
        match_dict = {}
        match_dict['index'] = index
        match_dict['home'] = home
        match_dict['away'] = away
        match_dict['consecutive'] = consecutive
        match_dict['datetime'] = datetime
        match_dict['day_of_week'] = day_of_week
        match_dict['weekend'] = weekend
        match_dict['holiday'] = holiday
        tot_matches_summary.append(match_dict)
        
        
    index += 1
            
            
            
        # print(club_matchs.iloc[k + 1])


In [52]:
myFile = open('total_matches_summary.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
writer.writerow(['index', 'home', 'away','consecutive', 'datetime', 'day_of_week','weekend', 'holiday'])
for row in tot_matches_summary:
    writer.writerow(row.values())
myFile.close()

In [42]:
total_matches[82]

{'index': 83,
 'home': '키움',
 'away': 'SSG',
 'consecutive': 1,
 'match_date': datetime.datetime(2023, 5, 5, 14, 0),
 'holiday': 2,
 'day_of_week': '금'}

In [43]:
import csv
f = open('match_rslt.csv', 'w', newline = '', encoding='utf8')
wr = csv.writer(f)

for line in total_matches:
    wr.writerow(line.values())
f.close()

{'index': 1, 'home': 'SSG', 'away': 'KIA', 'consecutive': 1, 'match_date': datetime.datetime(2023, 4, 1, 14, 0), 'holiday': 1, 'day_of_week': '토'}
{'index': 2, 'home': 'SSG', 'away': 'KIA', 'consecutive': 2, 'match_date': datetime.datetime(2023, 4, 2, 14, 0), 'holiday': 1, 'day_of_week': '일'}
{'index': 3, 'home': 'SSG', 'away': '롯데', 'consecutive': 1, 'match_date': datetime.datetime(2023, 4, 4, 18, 30), 'holiday': 0, 'day_of_week': '화'}
{'index': 4, 'home': 'SSG', 'away': '롯데', 'consecutive': 2, 'match_date': datetime.datetime(2023, 4, 5, 18, 30), 'holiday': 0, 'day_of_week': '수'}
{'index': 5, 'home': 'SSG', 'away': '롯데', 'consecutive': 3, 'match_date': datetime.datetime(2023, 4, 6, 18, 30), 'holiday': 0, 'day_of_week': '목'}
{'index': 6, 'home': 'SSG', 'away': 'NC', 'consecutive': 1, 'match_date': datetime.datetime(2023, 4, 14, 18, 30), 'holiday': 0, 'day_of_week': '금'}
{'index': 7, 'home': 'SSG', 'away': 'NC', 'consecutive': 2, 'match_date': datetime.datetime(2023, 4, 15, 17, 0), 'hol

## Calendar
"calendar": [
    {
      "index": 0,
      "datetime": "2021-04-02 00:00:00",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 0
    },


## initial plan

```
  {
      "index": 0,
      "datetime": "2021-04-02 00:00:00",
      "home": "두산",
      "away": "한화",
      "consecutive": 2.0,
      "holidays": 2.0,
    },
```
